In [1]:
import pickle
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import TensorBoard
from sklearn.metrics import accuracy_score
import datetime
import numpy as np
import pandas as pd

### loading the scaler,one hot encoded geo file, label encoded gender file,and the ANN trained model itself

In [2]:
model=load_model('model.h5')

with open('label_encode_geography.pkl','rb')as f:
    onehot_encode_geo=pickle.load(f)


with open('label_encode_gender.pkl','rb')as f:
    label_encode_gender=pickle.load(f)


with open('scaler.pkl','rb')as f:
    scaler=pickle.load(f)

with open('X_test_scaled.pkl','rb')as f:
    X_test_scaled=pickle.load(f)


with open('y_test.pkl','rb')as f:
    y_test=pickle.load(f)

### evaluate the performance on X_test

In [3]:
# log_dir = "tensoboard_logs/predictions/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
# tensorboard_callback = TensorBoard(log_dir=log_dir,histogram_freq=1)
# y_pred_prob = model.predict(X_test_scaled, callbacks=[tensorboard_callback])
# y_pred = (y_pred_prob > 0.5).astype(int)
# accuracy = accuracy_score(y_test, y_pred)
# print(f"Test Accuracy: {accuracy:.4f}")

In [4]:
# # Log test accuracy
# with tf.summary.create_file_writer(log_dir).as_default():
#     tf.summary.scalar('Test Accuracy', accuracy, step=1)

### Evaluate the performance on a random data

In [5]:
input_data = {
    'CreditScore': 450,          # Low credit score, indicating financial stress
    'Geography': 'Germany',      # Varies by region; some areas may show higher churn
    'Gender': 'Female',          # Gender can influence churn, though less directly
    'Age': 55,                   # Older customers may churn if dissatisfied
    'Tenure': 1,                 # Short tenure suggests less loyalty
    'Balance': 0,                # No balance, indicating inactivity
    'NumOfProducts': 1,          # Fewer products, less engagement
    'HasCrCard': 0,              # No credit card, less financial tie
    'IsActiveMember': 0,         # Inactive member, key churn indicator
    'EstimatedSalary': 25000     # Lower salary, potential financial pressure
}

### converting input data into a pandas dataframe

In [6]:
input_df=pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,450,Germany,Female,55,1,0,1,0,0,25000


In [7]:
geo_input = input_df[['Geography']]
geo_encoded=onehot_encode_geo.transform(geo_input).toarray()
geo_encoded_df=pd.DataFrame(geo_encoded,columns=onehot_encode_geo.get_feature_names_out(['Geography']))


In [8]:
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,0.0,1.0,0.0


In [9]:
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,450,Germany,Female,55,1,0,1,0,0,25000


### connverting the gender column of the input df to a label encoded version using transform

In [10]:
input_df['Gender']=label_encode_gender.transform(input_df['Gender'])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,450,Germany,0,55,1,0,1,0,0,25000


### drop the Geography column from the input df and concatenate the encoded df of geography 

In [11]:
input_df=pd.concat([input_df.drop('Geography',axis=1),geo_encoded_df],axis=1)
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,450,0,55,1,0,1,0,0,25000,0.0,1.0,0.0


### scale the values of input_df

In [12]:
scaled_input=scaler.transform(input_df)
scaled_input

array([[-2.0783899 , -1.10191942,  1.52508472, -1.38666414, -1.22605881,
        -0.91025649, -1.55996038, -1.030206  , -1.30348532, -0.99825153,
         1.72916886, -0.57773517]])

### prediction


In [13]:
predcition=model.predict(scaled_input)
predcition

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step


array([[0.98158234]], dtype=float32)

In [14]:
prediction_probability=predcition[0][0]

if prediction_probability >0.5:
    print('The customer is likely to leave')

else:
    print('The customer is not likely to leave')

The customer is likely to leave
